In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# File paths
train_csv_path = 'Train.csv'
test_csv_path = 'Test.csv'
meta_csv_path = 'Meta.csv'
train_images_dir = 'Train/'  # Directory containing training images
test_images_dir = 'Test/'    # Directory containing test images

# Load CSV files
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
meta_df = pd.read_csv(meta_csv_path)

# Load and preprocess data
def load_data(df, images_dir):
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = os.path.join(images_dir, row['Path'])
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))  # Resize to match input size
            img = img_to_array(img)
            img = img / 255.0  # Normalize the image
            images.append(img)
            labels.append(row['ClassId'])
    return np.array(images), np.array(labels)

# Load training and test data
train_images, train_labels = load_data(train_df, train_images_dir)
test_images, test_labels = load_data(test_df, test_images_dir)

# Convert labels to one-hot encoding
num_classes = len(np.unique(train_labels))
train_labels = to_categorical(train_labels, num_classes=num_classes)
test_labels = to_categorical(test_labels, num_classes=num_classes)

# Split data for validation
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Save the model
model.save('traffic_sign_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {accuracy:.4f}')



In [ ]:
# Function to preprocess and predict the traffic sign
def predict_traffic_sign(image_path):
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (64, 64))  # Resize to match input size
    img = img_to_array(img)
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    # Predict the class
    prediction = model.predict(img)
    class_id = np.argmax(prediction, axis=1)[0]

    return class_id

# Example usage
image_path = 'path/to/your/image.png'  # Update with the path to your image
predicted_class_id = predict_traffic_sign(image_path)

print(f'Predicted Class ID: {predicted_class_id}')